<a href="https://colab.research.google.com/github/cbonnin88/Equilibrium/blob/main/Deep_dive_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import polars as pl
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
from google.colab import auth

In [3]:
auth.authenticate_user()
project_id = 'project-equilibrium-474307'
client = bigquery.Client(project_id)
print('Authentication Successfull')

Authentication Successfull


In [4]:
master_query = f"""
WITH LatestPerformance AS (
  SELECT
    employee_id,
    performance_score,
    potential_score,
    ROW_NUMBER() OVER(PARTITION BY employee_id ORDER BY review_date DESC) AS rn
  FROM
    `project-equilibrium-474307.hr_analytics.performance`
),
LatestCompensation AS (
  SELECT
    employee_id,
    base_salary,
    bonus,
    stock_options,
    ROW_NUMBER() OVER(PARTITION BY employee_id ORDER BY effective_date DESC) AS rn
  FROM
    `project-equilibrium-474307.hr_analytics.compensation`
)
SELECT
  e.employee_id,
  e.gender,
  e.department,
  e.job_level,
  e.hire_date,
  e.termination_date,
  lc.base_salary,
  lc.bonus,
  lc.stock_options,
  lp.performance_score,
  lp.potential_score,
  CASE WHEN e.termination_date IS NOT NULL THEN 1 ELSE 0 END AS is_attrited,
  DATE_DIFF(COALESCE(e.termination_date, '2025-10-16'), e.hire_date, DAY) AS tenure_in_days
FROM
  `project-equilibrium-474307.hr_analytics.employees` AS e
LEFT JOIN LatestCompensation AS lc
  ON e.employee_id = lc.employee_id AND lc.rn = 1
LEFT JOIN LatestPerformance AS lp
  ON e.employee_id = lp.employee_id AND lp.rn = 1
WHERE
  lc.base_salary IS NOT NULL AND lp.performance_score IS NOT NULL;
"""

In [5]:
pandas_df = client.query(master_query).to_dataframe()
df_people = pl.from_pandas(pandas_df)

In [6]:
print(f'Data Successfully Extracted. Shape:{df_people.shape}')

Data Successfully Extracted. Shape:(2500, 13)


In [7]:
df_people.head()

employee_id,gender,department,job_level,hire_date,termination_date,base_salary,bonus,stock_options,performance_score,potential_score,is_attrited,tenure_in_days
i64,str,str,str,date,date,f64,i64,i64,i64,i64,i64,i64
8273,"""Male""","""Finance""","""T1""",2021-02-16,null,132800.0,8951,0,5,2,0,1703
8790,"""Non-Binary""","""Sales""","""T1""",2025-07-18,2025-07-26,96000.0,15017,2000,5,3,1,8
7634,"""Female""","""Data""","""T5""",2023-09-06,2025-01-25,36800.0,7201,2000,2,2,1,507
3829,"""Female""","""Product""","""T1""",2024-05-25,2024-06-27,117600.0,12103,500,1,2,1,33
4618,"""Non-Binary""","""Product""","""T2""",2024-07-11,null,98700.0,9211,1000,3,5,0,462


# *Compensation Equity Box Plot*

In [9]:
fig_box = px.box(
    df_people.to_pandas(),
    x='department',
    y='base_salary',
    color='gender',
    title='Compensation Distribution by Department and Gender',
    labels={'base_salary':'Base Salary (€)','department':'Department','gender':'Gender'},
    notched=True # Adds a 'notch' to show the confidence interval for the median
)

fig_box.show()

# **Tenure & Attrition**

In [10]:
# Creating a new column in Polars for 'Attrition Status'
df_with_status = df_people.with_columns(
    pl.when(pl.col('is_attrited')==1)
    .then(pl.lit('Left'))
    .otherwise(pl.lit('Stayed'))
    .alias('Attrition Status')
)

display(df_with_status)

employee_id,gender,department,job_level,hire_date,termination_date,base_salary,bonus,stock_options,performance_score,potential_score,is_attrited,tenure_in_days,Attrition Status
i64,str,str,str,date,date,f64,i64,i64,i64,i64,i64,i64,str
8273,"""Male""","""Finance""","""T1""",2021-02-16,null,132800.0,8951,0,5,2,0,1703,"""Stayed"""
8790,"""Non-Binary""","""Sales""","""T1""",2025-07-18,2025-07-26,96000.0,15017,2000,5,3,1,8,"""Left"""
7634,"""Female""","""Data""","""T5""",2023-09-06,2025-01-25,36800.0,7201,2000,2,2,1,507,"""Left"""
3829,"""Female""","""Product""","""T1""",2024-05-25,2024-06-27,117600.0,12103,500,1,2,1,33,"""Left"""
4618,"""Non-Binary""","""Product""","""T2""",2024-07-11,null,98700.0,9211,1000,3,5,0,462,"""Stayed"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
5970,"""Non-Binary""","""Human Resources""","""T2""",2021-10-30,null,98500.0,10503,2000,3,4,0,1447,"""Stayed"""
3563,"""Non-Binary""","""Human Resources""","""T3""",2020-10-30,null,79700.0,14736,500,4,1,0,1812,"""Stayed"""
2643,"""Male""","""Data""","""T3""",2023-03-18,null,34600.0,6605,2000,1,3,0,943,"""Stayed"""


In [12]:
fig_hist = px.histogram(
    df_with_status.to_pandas(),
    x='tenure_in_days',
    color='Attrition Status',
    barmode='overlay', # Lays one histogram on top of the other
    nbins=50,
    title='Employee Tenure Distribution (Stayed vs. Left)'
)

fig_hist.show()

# **Pay-for-Performance**

In [15]:
import numpy as np

df_jittered = df_people.with_columns(
    (pl.col('performance_score') + (pl.lit(0.1) * (pl.lit(2) * pl.Series(np.random.rand(len(df_people))) - pl.lit(1))))
    .alias('performance_score_jittered')
)

display(df_jittered)

employee_id,gender,department,job_level,hire_date,termination_date,base_salary,bonus,stock_options,performance_score,potential_score,is_attrited,tenure_in_days,performance_score_jittered
i64,str,str,str,date,date,f64,i64,i64,i64,i64,i64,i64,f64
8273,"""Male""","""Finance""","""T1""",2021-02-16,null,132800.0,8951,0,5,2,0,1703,4.959056
8790,"""Non-Binary""","""Sales""","""T1""",2025-07-18,2025-07-26,96000.0,15017,2000,5,3,1,8,4.93299
7634,"""Female""","""Data""","""T5""",2023-09-06,2025-01-25,36800.0,7201,2000,2,2,1,507,1.9308
3829,"""Female""","""Product""","""T1""",2024-05-25,2024-06-27,117600.0,12103,500,1,2,1,33,1.062314
4618,"""Non-Binary""","""Product""","""T2""",2024-07-11,null,98700.0,9211,1000,3,5,0,462,2.98577
…,…,…,…,…,…,…,…,…,…,…,…,…,…
5970,"""Non-Binary""","""Human Resources""","""T2""",2021-10-30,null,98500.0,10503,2000,3,4,0,1447,2.942768
3563,"""Non-Binary""","""Human Resources""","""T3""",2020-10-30,null,79700.0,14736,500,4,1,0,1812,4.030684
2643,"""Male""","""Data""","""T3""",2023-03-18,null,34600.0,6605,2000,1,3,0,943,0.918945


In [16]:
fig_scatter = px.scatter(
    df_jittered.to_pandas(),
    x='performance_score_jittered',
    y='base_salary',
    color='job_level',
    opacity=0.5, # Make points semi-transparent to see density
    title='Pay-for-Performance: Salary vs Performance Score',
    labels={'performance_score_jittered':'Performance Score','base_salary':'Base Salary (€)'},
    trendline='ols'
)

fig_scatter.show()

# **Company Composition**

In [18]:
sunburst_data = df_with_status.group_by(
    ['department','job_level','Attrition Status']
).agg(
    pl.len().alias('count')
)

display(sunburst_data)

department,job_level,Attrition Status,count
str,str,str,u32
"""Data""","""T2""","""Stayed""",72
"""Sales""","""T1""","""Stayed""",71
"""Human Resources""","""T1""","""Stayed""",61
"""Sales""","""T4""","""Left""",7
"""Engineering""","""T3""","""Stayed""",61
…,…,…,…
"""Engineering""","""T3""","""Left""",16
"""Sales""","""T3""","""Stayed""",72
"""Finance""","""T5""","""Left""",17


In [20]:
fig_sunburst = px.sunburst(
    sunburst_data.to_pandas(),
    path=['department','job_level','Attrition Status'],
    values='count',
    title='Company Composition by Department, Job Level and Attrition'
)

fig_sunburst.show()